In [2]:
# -*- coding: utf-8 -*-
import string  #用來處理字串
import pickle,re,sys,os,re,time
import math, numpy
from math import sqrt
from numpy import *

#-------------------------- function --------------------------
#相似度計算
def cosine_distance(u, v):
    """
    Returns the cosine of the angle between vectors v and u. This is equal to
    u.v / |u||v|.
    """
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v)))

#log2轉換
def log2(n):
    num = math.log(n) / math.log(2)
    return num

#讀取檔案內資料
def openFileData(fileName):
    TempFile = open(fileName)
    data = TempFile.read()
    TempFile.close()
    return data


#讀取檔案內資料成list
def fileDataList(fileName):
    TempFile = open(fileName)
    data = TempFile.read()
    TempFile.close()
    data = data.split('\n')
    return data


#去除list內空白字串
def deListNull(listName):
    for x in range(listName.count('')):
        listName.pop(listName.index(''))

#計算query tfidf
def queryTfidf(str):
	
    query = str.split(',')
    deListNull(query)
    qtf = zeros(len(dim))
    qtfidf=zeros(len(dim))
    
    for word in dim:
        if word in query:
            qtf[dim.index(word)]=query.count(word)
            
    for word in dim:
        if word in query:
            qtfidf[dim.index(word)] = multiply(qtf[dim.index(word)],idf[word])
    return qtfidf

#計算query和document相似度   
def qdSim(q):
    for fileName in fileList:
        openFileName = path+fileName
        text = openFileData(openFileName)
        text = text.replace("\n"," ")
        text = text.split(' ')
        deListNull(text)
        #將tfidf dictionary型別轉成list 
        d=zeros(len(dim))
        for i in range(len(dim)):
            word=dim[i]
            if word in text:
               d[i]=tfidf[fileName][word]
            else:
               d[i]=0
        print (fileName, "=", cosine_distance(q,d))
        
#-------------------------- Code --------------------------
if __name__=='__main__':

    path = "./data/"
    fileList = os.listdir(path)
    dim = [] #VSM維度詞彙
    tf = {}
    idf = {}
    tfidf = {}
    wordCount = {}


    #計算所有文章的詞彙取出VSM維度
    for fileName in fileList:
        tf[fileName] = {} #建立tf字典內該篇文章的空間
        openFileName = path+fileName
        text = openFileData(openFileName)
        text = text.replace("\n"," ")
        text = text.split(' ')
        deListNull(text)
    
        for word in text: 
            if word not in dim:
                dim.append(word)
                idf[word] = 0
                wordCount[word] = 0
    dim=sorted(dim)


    #計算所有文章的詞彙tf
    for fileName in fileList:
        openFileName = path+fileName
        text = openFileData(openFileName)
        text = text.replace("\n"," ")
        text = text.split(' ')
        deListNull(text)
        for word in dim:
            tf[fileName][word] =  text.count(word)
            wordCount[word] = wordCount[word] + text.count(word)

    #計算所有文章的詞彙idf
    for word in idf.items():
        a = float(len(fileList))/float(wordCount[word[0]])
        idf[word[0]] = log2(a) + 1      #IDF=log2 (n/nj ) + 1

    #存成CSV檔

    TempFile = open('TFIDF.csv','w')

    #儲存TF資料
    line = "TF,"
    for word in dim:
        line = line + word + ","

    line = line + "\n"
    TempFile.write(line)


    for text in tf.items():
        line = ''
        line = line + text[0] + ","
        for word in dim:
            line = line + str(text[1][word]) + ","
        line = line + "\n"
    
        TempFile.write(line)


    line = "IDF,"
    #儲存IDF資料
    for word in dim:
        line = line + str(idf[word]) + ","

    line = line + "\n\n\n"
    TempFile.write(line)


    #儲存TFIDF資料
    line = "TFIDF,"
    for word in dim:
        line = line + word + ","

    line = line + "\n"
    TempFile.write(line)

    for text in tf.items():
        tfidf[text[0]] = {}
        line = ''
        line = line + text[0] + ","
        #for word in dim:
        for word in dim:
            tfidf[text[0]][word] = text[1][word]*idf[word]#計算tfidf
            line = line + str(tfidf[text[0]][word]) + ","
        line = line + "\n"
    
        TempFile.write(line)

    TempFile.close()
	
    #計算query(d8~d10)和文章集(d1~d7)相似度
    d8='T11,T12,T13,T14,T15'
    d9='T3,T4,T13'
    d10='T5,T12,T15'
    print ("d8=",d8)
    qtfidf=queryTfidf(d8)
    qdSim(qtfidf)
    print ("d9=",d9)
    qtfidf=queryTfidf(d9)
    qdSim(qtfidf)
    print ("d10=",d10)
    qtfidf=queryTfidf(d10)
    qdSim(qtfidf)


d8= T11,T12,T13,T14,T15
D1.txt = 0.0
D2.txt = 0.0
D3.txt = 0.0
D4.txt = 0.20200699008968323
D5.txt = 0.16941673287541414
D6.txt = 0.45623101455117604
D7.txt = 0.8112991142216643
d9= T3,T4,T13
D1.txt = 0.8385303511564246
D2.txt = 0.0
D3.txt = 0.0
D4.txt = 0.2394885001357971
D5.txt = 0.20085126379154963
D6.txt = 0.0
D7.txt = 0.0
d10= T5,T12,T15
D1.txt = 0.13040259948139088
D2.txt = 0.13850274659004772
D3.txt = 0.137595109815053
D4.txt = 0.3681425762317379
D5.txt = 0.1858400129307935
D6.txt = 0.1928022341427289
D7.txt = 0.5765076082842013
